In [2]:
import load_env
import src.graphql.models.recipe_models_autogen as rma
from sqlalchemy import select
from src.graphql.db.session import get_session
from src.graphql.scalars.recipe_scalar import NEI_Item, NEI_Fluid

In [3]:
# Test joins
recipe_id = 'r~--eEYPHsN5GqWygrEHan4A=='

# stmt = select(rma.Fluid, rma.RecipeFluidGroup.fluid_inputs_key, rma.FluidGroupFluidStacks.fluid_stacks_amount) \
#     .join(rma.FluidGroupFluidStacks, rma.FluidGroupFluidStacks.fluid_stacks_fluid_id == rma.Fluid.id) \
#     .join(rma.RecipeFluidGroup, rma.RecipeFluidGroup.fluid_inputs_id == rma.FluidGroupFluidStacks.fluid_group_id) \
#     .filter(rma.RecipeFluidGroup.recipe_id == recipe_id)
stmt = select(rma.Item, rma.RecipeItemOutputs) \
        .join(rma.RecipeItemOutputs, rma.RecipeItemOutputs.item_outputs_value_item_id == rma.Item.id) \
        .filter(rma.RecipeItemOutputs.recipe_id == recipe_id)
print(stmt)

SELECT item.id, item.image_file_path, item.internal_name, item.item_damage, item.item_id, item.localized_name, item.max_damage, item.max_stack_size, item.mod_id, item.nbt, item.tooltip, item.unlocalized_name, recipe_item_outputs.recipe_id, recipe_item_outputs.item_outputs_value_probability, recipe_item_outputs.item_outputs_value_stack_size, recipe_item_outputs.item_outputs_key, recipe_item_outputs.item_outputs_value_item_id 
FROM item JOIN recipe_item_outputs ON recipe_item_outputs.item_outputs_value_item_id = item.id 
WHERE recipe_item_outputs.recipe_id = :recipe_id_1


In [6]:
def _prepORMDict(item, rename={}, exclude=[], include={}):
    d = dict(item.__dict__)
    d.pop('_sa_instance_state')

    for k, v in rename.items():
        d[v] = d.pop(k)
    for k in exclude:
        d.pop(k)
    d.update(include)
    
    return d


async with get_session() as session:
    rows = await session.execute(stmt)
    item_inputs = []
    for r in rows:
        itemORM, itemOutputsORM = r
        findict = {}
        findict.update(_prepORMDict(itemORM))
        print(itemOutputsORM.__dict__)
        findict.update(_prepORMDict(
            itemOutputsORM,
            exclude=['recipe_id', 'item_outputs_value_item_id'],
            rename={
                'item_outputs_value_stack_size': 'stack_size',
                'item_outputs_value_probability': 'output_probability',
                'item_outputs_key': 'position',
            }
        ))
        item_inputs.append(
            NEI_Item(**findict)
        )

    # print(result)
    # for r in result:
    #     print(r.__dict__)

    print(item_inputs)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7fb8a8c59c50>, 'item_outputs_value_stack_size': 1, 'item_outputs_value_item_id': 'i~gregtech~gt.metaitem.01~32601', 'item_outputs_value_probability': 1.0, 'recipe_id': 'r~--eEYPHsN5GqWygrEHan4A==', 'item_outputs_key': 0}


KeyError: 'item_outputs_value_item_id'